In [1]:
import numpy as np
import pandas as pd
#dataset import
df = pd.read_csv("../Data/data_5_nodes_yen.csv") #You need to change #directory accordingly
df.head()

,"X(1, 2)","X(1, 3)","X(1, 4)","X(2, 4)","X(2, 5)","X(3, 4)","X(3, 5)","X(4, 5)","Y1(1, 2)","Y1(1, 3)",...,"Y3(3, 5)","Y3(4, 5)","Y4(1, 2)","Y4(1, 3)","Y4(1, 4)","Y4(2, 4)","Y4(2, 5)","Y4(3, 4)","Y4(3, 5)","Y4(4, 5)"
0,40,30,50,40,90,90,70,70,0,1,...,0,0,1,0,0,1,0,0,0,1
1,10,30,20,60,70,90,80,80,1,0,...,1,0,1,0,0,1,0,0,0,1
2,50,60,70,80,90,90,50,10,0,0,...,0,0,1,0,0,1,0,0,0,1
3,30,70,20,10,70,20,70,60,0,0,...,0,0,1,0,0,1,0,0,0,1
4,60,80,10,10,20,40,60,60,0,0,...,0,0,0,0,1,0,0,1,1,0


In [2]:
X = df.iloc[:20000,:8].values
y = df.iloc[:20000,8:].values

In [3]:
X.shape

(20000, 8)

In [4]:
y.shape

(20000, 32)

In [5]:
from sklearn.model_selection import train_test_split
X1,X2,y_train,y_test = train_test_split(X,y,test_size = 0.3)

In [6]:
X2[1]

array([10, 40, 50, 40, 70, 20, 40, 10], dtype=int64)

In [7]:
y_test[1]

array([0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0], dtype=int64)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X1)
X_test= sc.fit_transform(X2)

In [9]:
X_train = np.repeat(X_train[:, np.newaxis, : ], 4, axis=1)
X_train.shape

(14000, 4, 8)

In [10]:
X_test = np.repeat(X_test[:, np.newaxis, : ], 4, axis=1)
X_test.shape

(6000, 4, 8)

In [11]:
y_train[0]

array([0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 1], dtype=int64)

In [12]:
y_train_2 = np.reshape(y_train,(y_train.shape[0], 4, 8))
y_train_2.shape

(14000, 4, 8)

In [13]:
y_train_2[0]

array([[0, 1, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 1, 0, 1],
       [1, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 1]], dtype=int64)

In [14]:
y_test_2 = np.reshape(y_test,(y_test.shape[0], 4, 8))
y_test_2.shape

(6000, 4, 8)

In [15]:
X2[1,:]

array([10, 40, 50, 40, 70, 20, 40, 10], dtype=int64)

In [16]:
#Dependencies
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,SimpleRNN,TimeDistributed
# Neural network
model = Sequential()
model.add(SimpleRNN(64, input_shape=(4, 8), 
                    return_sequences=True))
model.add(SimpleRNN(64, return_sequences=True))
model.add(TimeDistributed(Dense(8, activation='sigmoid')))

In [17]:
model.compile(loss='BinaryCrossentropy', optimizer='adam')

In [18]:
model.fit(X_train, y_train_2, epochs=100, batch_size=128)

Epoch 1/100
110/110 [==============================] - 2s 3ms/step - loss: 0.5622
Epoch 2/100
110/110 [==============================] - 0s 3ms/step - loss: 0.4406
Epoch 3/100
110/110 [==============================] - 0s 3ms/step - loss: 0.3835
Epoch 4/100
110/110 [==============================] - 0s 3ms/step - loss: 0.3467
Epoch 5/100
110/110 [==============================] - 0s 3ms/step - loss: 0.3175
Epoch 6/100
110/110 [==============================] - 0s 3ms/step - loss: 0.2922
Epoch 7/100
110/110 [==============================] - 0s 3ms/step - loss: 0.2709
Epoch 8/100
110/110 [==============================] - 0s 3ms/step - loss: 0.2525
Epoch 9/100
110/110 [==============================] - 0s 4ms/step - loss: 0.2370
Epoch 10/100
110/110 [==============================] - 0s 4ms/step - loss: 0.2241
Epoch 11/100
110/110 [==============================] - 0s 4ms/step - loss: 0.2117
Epoch 12/100
110/110 [==============================] - 0s 3ms/step - loss: 0.2019
Epoch 13/100


110/110 [==============================] - 0s 3ms/step - loss: 0.0403
Epoch 100/100
110/110 [==============================] - 0s 3ms/step - loss: 0.0402


In [19]:
len(model.layers)

3

In [20]:
pred=model.predict(X_train)

438/438 [==============================] - 1s 1ms/step


In [21]:
pred.shape

(14000, 4, 8)

In [22]:
pred[0]

array([[1.37066923e-03, 9.99561727e-01, 5.08717312e-05, 1.70270680e-04,
        3.91734429e-05, 9.28098143e-06, 9.99580860e-01, 1.06844830e-03],
       [4.85797584e-01, 3.04412365e-01, 4.47596540e-06, 6.99410215e-03,
        6.12686528e-03, 6.59138680e-01, 2.52188230e-03, 7.11565971e-01],
       [9.98652518e-01, 4.18144986e-02, 4.49467552e-06, 2.31227875e-02,
        9.35615718e-01, 2.89773103e-03, 1.27369698e-04, 1.04616985e-01],
       [9.92276371e-01, 3.20560252e-03, 3.39233573e-03, 9.94716704e-01,
        2.44558696e-03, 7.46014901e-03, 7.55366962e-03, 9.98599946e-01]],
      dtype=float32)

In [23]:
y_train_2[0]

array([[0, 1, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 1, 0, 1],
       [1, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 1]], dtype=int64)

In [24]:
edges = [[1, 2], [1, 3], [1, 4], [2, 4], [2, 5], [3, 4], [3, 5], [4, 5]]

In [25]:
import copy
def cost_path(x,y):
    cost = 0
    path = []
    for i in range(8):
        if y[i]==1:
            path.append(edges[i])
            cost+= x[i]
    if(len(path)!=0):
        path_2 = copy.deepcopy(path)
        path_nodes = [1]
        if(path[0][0]!=1):
            cost = 0
        else:
            path_nodes.append(path[0][1])
            del path_2[0]
            condition = True
            while(len(path_2)!=0 and condition):
                condition = False
                for i in range(len(path_2)):
                    if(path_nodes[-1] in path_2[i]):
                        path_2[i].remove(path_nodes[-1])
                        path_nodes.append(path_2[i][0])
                        path_2.pop(i)
                        condition = True
                        break
        if(path_nodes[-1]!=5 or len(path_2)!=0):
            cost = 0
    else:
        cost = 0
    return cost

In [26]:
cost_path(X1[1],y_train[1])

130

In [27]:
accuracy = 0
for i in range(pred.shape[0]):
    output = copy.deepcopy(pred[i][3])
    condition = True
    for j in range(len(output)):
        output[j] = round(output[j])
    if(i%10000 == 0):
        print(output, cost_path(X1[i],output))
    if(cost_path(X1[i],output) != cost_path(X1[i],y_train_2[i][3])):
        condition = False
    if condition:
        accuracy += 1
accuracy / pred.shape[0] * 100

[1. 0. 0. 1. 0. 0. 0. 1.] 110
[0. 0. 1. 0. 0. 1. 1. 0.] 150


93.63571428571429

In [28]:
pred_test=model.predict(X_test)

188/188 [==============================] - 0s 1ms/step


In [29]:
accuracy_test = 0
for i in range(pred_test.shape[0]):
    output = copy.deepcopy(pred_test[i][3])
    condition = True
    for j in range(len(output)):
        output[j] = round(output[j])
    if(cost_path(X2[i],output)!=cost_path(X2[i],y_test_2[i][3])):
        condition = False
    if condition:
        accuracy_test += 1
accuracy_test / pred_test.shape[0] * 100

87.58333333333333

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 4, 64)             4672      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 4, 64)             8256      
                                                                 
 time_distributed (TimeDistr  (None, 4, 8)             520       
 ibuted)                                                         
                                                                 
Total params: 13,448
Trainable params: 13,448
Non-trainable params: 0
_________________________________________________________________
